# Season tables reconstruction

In [1]:
import pandas as pd

df = pd.read_parquet('data/serie_a_matches_processed.parquet')

In [2]:
df.tail()

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,pkatt,season,team,opp captain,opp sh,opp sot,opp dist,opp fk,opp pk,opp pkatt
1946,2020-09-20,18:00,Serie A,1,Sun,Home,D,1,1,Cagliari,...,0,2020,Sassuolo,João Pedro,8,4,15.9,1,0,0
1947,2020-09-20,15:00,Serie A,1,Sun,Home,W,4,1,Crotone,...,0,2020,Genoa,Alex Cordaz,13,3,18.8,0,0,0
1948,2020-09-20,12:30,Serie A,1,Sun,Home,L,0,2,Napoli,...,0,2020,Parma,Lorenzo Insigne,17,6,19.4,0,0,0
1949,2020-09-19,20:45,Serie A,1,Sat,Home,W,3,0,Roma,...,0,2020,Hellas Verona,Lorenzo Pellegrini,21,4,NaN,0,0,0
1950,2020-09-19,18:00,Serie A,1,Sat,Home,W,1,0,Torino,...,0,2020,Fiorentina,Andrea Belotti,6,3,18.7,1,0,0


In [ ]:
# get unique seasons
seasons = df['season'].unique()
seasons

array([2025, 2024, 2023, 2022, 2021, 2020])

In [4]:
df[df['season'] == 2022]['round'].unique().size

39

In 2022 round with value '0' was a relegation play-off between Spezia and Verona

In [ ]:
!pip3 install great_tables

## Calculate points

In [5]:
from great_tables import GT

season = 2020

df_2020 = df[df['season'] == season]

# create empty dataframe for points
points_df = pd.DataFrame(columns=['team', 'points', 'w', 'd', 'l'])
points_df['team'] = pd.unique(df_2020[['team']].values.ravel('K'))
points_df[['points', 'w', 'd', 'l']] = 0
# points_df

## calculate points for each match
def calculate_points(row) -> tuple[int, int]:
  if row['result'] == 'L': #home team lost - 0 pts
    points_df.loc[points_df['team'] == row['opponent'], 'points'] += 3
    points_df.loc[points_df['team'] == row['opponent'], 'w'] += 1
    points_df.loc[points_df['team'] == row['team'], 'l'] += 1
  elif row['result'] == 'W': #home team won - 3 pts
    points_df.loc[points_df['team'] == row['team'], 'points'] += 3
    points_df.loc[points_df['team'] == row['team'], 'w'] += 1
    points_df.loc[points_df['team'] == row['opponent'], 'l'] += 1
  elif row['result'] == 'D': #draw - 1 pt each
    points_df.loc[points_df['team'] == row['team'], 'points'] += 1
    points_df.loc[points_df['team'] == row['opponent'], 'points'] += 1
    points_df.loc[points_df['team'] == row['team'], 'd'] += 1
    points_df.loc[points_df['team'] == row['opponent'], 'd'] += 1

# apply function to each row
df_2020.apply(calculate_points, axis=1)

points_df.sort_values(by='points', ascending=False, inplace=True)
points_df

,team,points,w,d,l
6,Internazionale,91,28,7,3
12,Milan,79,24,7,7
0,Atalanta,78,23,9,6
18,Juventus,78,23,9,6
1,Napoli,77,24,5,9
10,Lazio,68,21,5,12
17,Roma,62,18,8,12
2,Sassuolo,62,17,11,10
7,Sampdoria,52,15,7,16
11,Hellas Verona,45,11,12,15


### Additional rules to determine order in table:
1. points scored
2. points scored in direct games
3. goals difference in direct games
4. goals difference in a season
5. bigger goals difference
6. more goals scored
7. more goals conceeded
8. higher place in fair play standings


## Calculate goals scored, conceeded and goals difference

In [6]:
# create empty dataframe for goals
goals_df = pd.DataFrame(columns=['team', 'goals_scored', 'goals_conceded', 'goals_difference', 'xg_for', 'xg_against'])
goals_df['team'] = pd.unique(df_2020[['team']].values.ravel('K'))
goals_df[['goals_scored', 'goals_conceded', 'goals_difference']] = 0
goals_df[['xg_for', 'xg_against']] = 0.0

def calulate_goals(row):
	if pd.isna(row['xg']):
		row['xg'] = 0.0
	if pd.isna(row['xga']):
		row['xga'] = 0.0
	goals_df.loc[goals_df['team'] == row['team'], 'goals_scored'] += row['gf']
	goals_df.loc[goals_df['team'] == row['team'], 'goals_conceded'] += row['ga']
	goals_df.loc[goals_df['team'] == row['team'], 'xg_for'] += row['xg']
	goals_df.loc[goals_df['team'] == row['team'], 'xg_against'] += row['xga']
	
	goals_df.loc[goals_df['team'] == row['opponent'], 'goals_scored'] += row['ga']
	goals_df.loc[goals_df['team'] == row['opponent'], 'goals_conceded'] += row['gf']
	goals_df.loc[goals_df['team'] == row['opponent'], 'xg_for'] += row['xga']
	goals_df.loc[goals_df['team'] == row['opponent'], 'xg_against'] += row['xg']

	goals_df.loc[goals_df['team'] == row['team'], 'goals_difference'] = goals_df['goals_scored'] - goals_df['goals_conceded']

# goals_df['goals_difference'] = goals_df['goals_scored'] - goals_df['goals_conceded']

df_2020.apply(calulate_goals, axis=1)

goals_df.sort_values(by='goals_difference', ascending=False, inplace=True)
goals_df


,team,goals_scored,goals_conceded,goals_difference,xg_for,xg_against
6,Internazionale,89,35,54,74.2,40.0
1,Napoli,86,41,43,67.8,42.6
18,Juventus,77,38,39,74.3,39.2
0,Atalanta,90,47,38,74.5,41.0
12,Milan,74,41,33,70.7,47.1
17,Roma,68,58,13,65.4,45.7
2,Sassuolo,64,56,8,58.3,55.6
10,Lazio,61,55,4,58.3,47.2
7,Sampdoria,52,54,1,45.5,56.2
11,Hellas Verona,46,48,-2,45.6,49.3


## Join points and goals dataframes

In [7]:
def join_points_and_goals(df_points: pd.DataFrame, df_goals: pd.DataFrame) -> pd.DataFrame:
  joined_df = df_points.merge(df_goals, on='team')
  joined_df.sort_values(by=['points', 'goals_difference'], ascending=False, inplace=True)
  joined_df.reset_index(drop=True, inplace=True)
  joined_df.index += 1
  return joined_df


join_points_and_goals(points_df, goals_df)

,team,points,w,d,l,goals_scored,goals_conceded,goals_difference,xg_for,xg_against
1,Internazionale,91,28,7,3,89,35,54,74.2,40.0
2,Milan,79,24,7,7,74,41,33,70.7,47.1
3,Juventus,78,23,9,6,77,38,39,74.3,39.2
4,Atalanta,78,23,9,6,90,47,38,74.5,41.0
5,Napoli,77,24,5,9,86,41,43,67.8,42.6
6,Lazio,68,21,5,12,61,55,4,58.3,47.2
7,Roma,62,18,8,12,68,58,13,65.4,45.7
8,Sassuolo,62,17,11,10,64,56,8,58.3,55.6
9,Sampdoria,52,15,7,16,52,54,1,45.5,56.2
10,Hellas Verona,45,11,12,15,46,48,-2,45.6,49.3


In [8]:
def calculate_table_for_round(df_matches: pd.DataFrame) -> pd.DataFrame:
    """
    Calculate league table based on matches provided.
    df_matches should contain all matches up to and including the target round.
    """
    teams = pd.unique(df_matches[['team', 'opponent']].values.ravel('K'))
    
    # Initialize table
    table = pd.DataFrame({
        'team': teams,
        'played': 0,
        'w': 0,
        'd': 0,
        'l': 0,
        'points': 0,
        'gf': 0,
        'ga': 0,
        'gd': 0,
        'xg_for': 0.0,
        'xg_against': 0.0
    })
    
    # Process each match
    for _, match in df_matches.iterrows():
        home_team = match['team']
        away_team = match['opponent']
        result = match['result']
        
        # Handle NaN values for xG
        xg = 0.0 if pd.isna(match['xg']) else match['xg']
        xga = 0.0 if pd.isna(match['xga']) else match['xga']
        
        # Update matches played
        table.loc[table['team'] == home_team, 'played'] += 1
        table.loc[table['team'] == away_team, 'played'] += 1
        
        # Update goals
        table.loc[table['team'] == home_team, 'gf'] += match['gf']
        table.loc[table['team'] == home_team, 'ga'] += match['ga']
        table.loc[table['team'] == away_team, 'gf'] += match['ga']
        table.loc[table['team'] == away_team, 'ga'] += match['gf']
        
        # Update xG
        table.loc[table['team'] == home_team, 'xg_for'] += xg
        table.loc[table['team'] == home_team, 'xg_against'] += xga
        table.loc[table['team'] == away_team, 'xg_for'] += xga
        table.loc[table['team'] == away_team, 'xg_against'] += xg
        
        # Update points and W/D/L
        if result == 'W':  # Home win
            table.loc[table['team'] == home_team, 'points'] += 3
            table.loc[table['team'] == home_team, 'w'] += 1
            table.loc[table['team'] == away_team, 'l'] += 1
        elif result == 'L':  # Away win
            table.loc[table['team'] == away_team, 'points'] += 3
            table.loc[table['team'] == away_team, 'w'] += 1
            table.loc[table['team'] == home_team, 'l'] += 1
        elif result == 'D':  # Draw
            table.loc[table['team'] == home_team, 'points'] += 1
            table.loc[table['team'] == away_team, 'points'] += 1
            table.loc[table['team'] == home_team, 'd'] += 1
            table.loc[table['team'] == away_team, 'd'] += 1
    
    # Calculate goal difference
    table['gd'] = table['gf'] - table['ga']
    
    # Sort by points, then goal difference, then goals scored
    table = table.sort_values(
        by=['points', 'gd', 'gf'], 
        ascending=[False, False, False]
    ).reset_index(drop=True)
    
    # Add position column
    table.insert(0, 'pos', range(1, len(table) + 1))
    
    return table


def generate_all_season_tables(df: pd.DataFrame) -> dict:
    """
    Generate tables for all seasons and all rounds.
    Returns a nested dictionary: {season: {round: table_dataframe}}
    """
    all_tables = {}
    seasons = sorted(df['season'].unique())
    
    for season in seasons:
        print(f"Processing season {season}...")
        season_data = df[df['season'] == season].copy()
        
        # Exclude round 0 (play-offs)
        season_data = season_data[season_data['round'] > 0]
        
        rounds = sorted(season_data['round'].unique())
        all_tables[season] = {}
        
        for current_round in rounds:
            # Get all matches up to and including current round
            matches_until_round = season_data[season_data['round'] <= current_round]
            
            # Calculate table
            table = calculate_table_for_round(matches_until_round)
            table['round'] = current_round
            table['season'] = season
            
            all_tables[season][current_round] = table
        
        print(f"  ✓ Generated {len(rounds)} tables for season {season}")
    
    return all_tables


# Generate all tables
season_tables = generate_all_season_tables(df)
print(f"\n✓ Total seasons processed: {len(season_tables)}")

Processing season 2020...
  ✓ Generated 38 tables for season 2020
Processing season 2021...
  ✓ Generated 38 tables for season 2021
Processing season 2022...
  ✓ Generated 38 tables for season 2022
Processing season 2023...
  ✓ Generated 38 tables for season 2023
Processing season 2024...
  ✓ Generated 38 tables for season 2024
Processing season 2025...
  ✓ Generated 5 tables for season 2025

✓ Total seasons processed: 6


In [15]:
season_tables[2025][3]

,pos,team,played,w,d,l,points,gf,ga,gd,xg_for,xg_against,round,season
0,1,Napoli,3,3,0,0,9,6,1,5,5.4,2.3,3,2025
1,2,Juventus,3,3,0,0,9,7,3,4,3.9,2.0,3,2025
2,3,Cremonese,3,2,1,0,7,5,3,2,3.1,4.6,3,2025
3,4,Udinese,3,2,1,0,7,4,2,2,2.8,2.8,3,2025
4,5,Milan,3,2,0,1,6,4,2,2,4.6,0.7,3,2025
5,6,Roma,3,2,0,1,6,2,1,1,3.5,1.8,3,2025
6,7,Atalanta,3,1,2,0,5,6,3,3,4.8,1.8,3,2025
7,8,Como,3,1,1,1,4,3,2,1,3.8,3.6,3,2025
8,9,Cagliari,3,1,1,1,4,3,2,1,2.7,4.4,3,2025
9,10,Torino,3,1,1,1,4,1,5,-4,2.5,4.4,3,2025


In [29]:
# Flatten the nested dictionary structure into a single DataFrame
all_tables_list = []

for season, rounds_dict in season_tables.items():
    for round_num, table_df in rounds_dict.items():
        all_tables_list.append(table_df)

# Concatenate all tables into one DataFrame
seasons_df = pd.concat(all_tables_list, ignore_index=True)

print(f"Total rows: {len(seasons_df):,}")
print(f"Seasons: {sorted(seasons_df['season'].unique())}")
print(f"\nFirst few rows:")
print(seasons_df.head())

# Save to parquet
seasons_df.to_parquet('data/serie_a_season_tables.parquet', index=False)
print("\n✓ Successfully saved to 'data/serie_a_season_tables.parquet'")

Total rows: 3,900
Seasons: [np.int64(2020), np.int64(2021), np.int64(2022), np.int64(2023), np.int64(2024), np.int64(2025)]

First few rows:
   pos            team  played  w  d  l  points  gf  ga  gd  xg_for  \
0    1  Internazionale       1  1  0  0       3   5   2   3     3.6   
1    2           Genoa       1  1  0  0       3   4   1   3     2.3   
2    3        Atalanta       1  1  0  0       3   4   1   3     1.8   
3    4        Juventus       1  1  0  0       3   3   0   3     2.0   
4    5   Hellas Verona       1  1  0  0       3   3   0   3     0.0   

   xg_against  round  season  
0         1.5      1    2020  
1         1.6      1    2020  
2         2.3      1    2020  
3         0.8      1    2020  
4         0.0      1    2020  

✓ Successfully saved to 'data/serie_a_season_tables.parquet'


# TODO

3. Implement the rest rules to determine table order
4. Make tables interactive if possible 

In [31]:
test = pd.read_parquet('data/serie_a_season_tables.parquet')
test.head()

,pos,team,played,w,d,l,points,gf,ga,gd,xg_for,xg_against,round,season
0,1,Internazionale,1,1,0,0,3,5,2,3,3.6,1.5,1,2020
1,2,Genoa,1,1,0,0,3,4,1,3,2.3,1.6,1,2020
2,3,Atalanta,1,1,0,0,3,4,1,3,1.8,2.3,1,2020
3,4,Juventus,1,1,0,0,3,3,0,3,2.0,0.8,1,2020
4,5,Hellas Verona,1,1,0,0,3,3,0,3,0.0,0.0,1,2020


## Loading and Reconstructing the Nested Dictionary Structure

To access the data in the same manner as before (as `season_tables[season][round]`), use this helper function:

In [ ]:
def load_season_tables(parquet_file='data/serie_a_season_tables.parquet'):
    """
    Load the flattened parquet file and reconstruct the nested dictionary structure.
    Returns: {season: {round: DataFrame}}
    """
    # Read the parquet file
    df = pd.read_parquet(parquet_file)
    
    # Reconstruct the nested dictionary
    season_tables_loaded = {}
    
    for season in sorted(df['season'].unique()):
        season_tables_loaded[season] = {}
        season_df = df[df['season'] == season]
        
        for round_num in sorted(season_df['round'].unique()):
            # Get the table for this specific season and round
            table = season_df[season_df['round'] == round_num].copy()
            
            season_tables_loaded[season][round_num] = table
    
    return season_tables_loaded


# Example usage:
loaded_tables = load_season_tables()

print(f"✓ Loaded {len(loaded_tables)} seasons")
print(f"Seasons available: {list(loaded_tables.keys())}")

# Access data the same way as before
print("\nExample - Table after round 3 of season 2025:")
loaded_tables[2025][3].head()

✓ Loaded 6 seasons
Seasons available: [np.int64(2020), np.int64(2021), np.int64(2022), np.int64(2023), np.int64(2024), np.int64(2025)]

Example - Table after round 3 of season 2025:


,pos,team,played,w,d,l,points,gf,ga,gd,xg_for,xg_against,round,season
3840,1,Napoli,3,3,0,0,9,6,1,5,5.4,2.3,3,2025
3841,2,Juventus,3,3,0,0,9,7,3,4,3.9,2.0,3,2025
3842,3,Cremonese,3,2,1,0,7,5,3,2,3.1,4.6,3,2025
3843,4,Udinese,3,2,1,0,7,4,2,2,2.8,2.8,3,2025
3844,5,Milan,3,2,0,1,6,4,2,2,4.6,0.7,3,2025
